В Примерах 1 и 2 мы познакомимся с вычислиением скорректированного коэффициента детерминации и его смыслом

В Примере 3 увидим, почему нельзя в линейной регрессии доверять экстраполяции

В Примере 4 изучим один из способов обнаружить нелинейную зависимость между фактором и целевой переменной

In [5]:
# импорт библиотек и датасета
%pylab inline
#%pip install --force-reinstall "scikit-learn==1.0.2"

import numpy as np
import pandas as pd
import math as mt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Пример 1

Вычислим обычный и скорректированный коэффициенты детерминации для разных моделей линейных регрессий на примере Boston Housing и посмотрим, как они будут различаться на обучающей и тестовой выборках.

In [20]:
#pip install --force-reinstall "scikit-learn==1.0.2"




In [15]:
# загрузим датасет Boston Housing


#dataset уданенный
#boston = datasets.load_boston()
#bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)


#dataset новый для работы кода. 
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
bostonDF = pd.DataFrame(raw_df, columns=raw_df.columns)


#y = boston.target

In [16]:
# добавим мусорный признак в датасет - случайный целочисленный вектор со значениями от 0 до 10
bostonDF['TRASH'] = pd.Series(np.random.randint(0,10,506))

In [17]:
# разделим датасет на обучающую и тестовую выборки
# параметр random_state обещает, что каждый раз, когда мы запустим команду, результат деления на train и test будет одинаковым
# вы можете проверить это, запустив блок несколько раз
# зададим размер тренировочной выборки 90%
X_train, X_test, y_train, y_test = train_test_split(bostonDF, y, train_size=0.9, random_state=151)
# любуемся на кусочек результата
X_train.head()

NameError: name 'y' is not defined

Создадим функцию regscores, которая будет вычислять обычные и исправленные коэффициенты детермианции для тренировочной и тестовой выборок

Ее параметры:

factors - список столбцов датасета, по которым мы будем прогонять регрессию

X_train, X_test, y_train, y_test - тренировочные и тестовые части датасета

In [0]:
def regscores(factors, X_train, X_test, y_train, y_test):
    
    # данные, на которых будет обучаться и тестироваться модель
    Data_train = X_train[factors]
    Data_test = X_test[factors]
    
    k = len(factors) # количество признаков (без константы)
    N_train = Data_train.shape[0] # размер обучающей выборки
    N_test = Data_test.shape[0] # размер тестовой вбыорки
    
    # обучим регрессию и интерсептом на тренировочной выборке
    # параметр fit_intercept =True можно не указывать, он установлен по умолчанию
    reg = LinearRegression(fit_intercept =True).fit(Data_train, y_train)

    # сделаем пердсказания на обучающей и тестовой выборках
    y_train_pred = reg.predict(Data_train)
    y_test_pred = reg.predict(Data_test)
    
    # создадим выходной вектор, в который будем записывать результат
    r2 = np.zeros(4)
    
    # R-squared для тренировочной выборки
    # (можно использовать reg.score, не импортируя отдельно r2_score)
    r2[0] = r2_score(y_train,y_train_pred)
    # R-squared adjusted для тренировочной выборки
    r2[1] = 1 - (1 - r2[0])*(N_train-1)/(N_train-k)
    
    # R-squared для тестовой выборки
    r2[2] = r2_score(y_test,y_test_pred)
    # R-squared adjusted для тестовой выборки
    r2[3] = 1 - (1 - r2[2])*(N_test-1)/(N_test-k)
    

    
    return r2

In [0]:
print('Модель с 2 факторами')
list_col=['CRIM', 'RM']
R2 = regscores(list_col, X_train, X_test, y_train, y_test)
print('R-squared для тренировочной выборки = ', R2[0])
print('R-squared adj для тренировочной выборки = ', R2[1])
print('R-squared для тестовой выборки = ', R2[2])
print('R-squared adj для тестовой выборки = ', R2[3])

Выводы:

1. Коэффициенты детерминации на тренировочной и тестовой выборке примерно одинаковые, а это значит, что модель не переобучилась
2. R-sq > R-sq adj
3. На тренировочной выборке разница между  R-sq и R-sq adj невелика. Так происходит, потому что размер выборки большой (около 450 точек), а факторов всего 2 - отношение (N-1)/(n-k) будет мало отличаться от 1, а значит и значения коэффициентов детерминации будут близки
4. Подумайте, почему на тестовой выборке R-sq и R-sq adj отличаются более заметно?

In [0]:
print('Модель с 2 факторами и мусорным фактором')
list_col=['CRIM', 'RM','TRASH']
R2 = regscores(list_col, X_train, X_test, y_train, y_test)
print('R-squared для тренировочной выборки = ', R2[0])
print('R-squared adj для тренировочной выборки = ', R2[1])
print('R-squared для тестовой выборки = ', R2[2])
print('R-squared adj для тестовой выборки = ', R2[3])

Выводы:

1. Коэффициенты детерминации на тренировочной и тестовой выборке примерно одинаковые, и это значит, что модель не переобучилась
2. R-sq < R-sq adj
3. R-sq на обеих выборках чуть выше, чем в модели без мусорного фактора
4. R-sq adj на тренировочной выборке отличаются совсем немного. Подумайте, почему?
5. R-sq adj на тестовой выборке заметно меньше, чем на тренировочной. Это значит, что R-sq adj "поймал" нас на добавлении "лишнего" фактора.

In [0]:
list_col=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD','TAX', 
       'PTRATIO', 'B', 'LSTAT']
R2  = regscores(list_col, X_train, X_test, y_train, y_test)
print('R-squared для тренировочной выборки = ', R2[0])
print('R-squared adj для тренировочной выборки = ', R2[1])
print('R-squared для тестовой выборки = ', R2[2])
print('R-squared adj для тестовой выборки = ', R2[3])

In [0]:
list_col=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT','TRASH']
R2 = regscores(list_col, X_train, X_test, y_train, y_test)
print('R-squared для тренировочной выборки = ', R2[0])
print('R-squared adj для тренировочной выборки = ', R2[1])
print('R-squared для тестовой выборки = ', R2[2])
print('R-squared adj для тестовой выборки = ', R2[3])

Выводы для моделей с большим количеством факторов:

1. Взяли в модель все имеющиеся факторы. R-sq на train и test отличаются незначительно, а R-sq adj - существенно, около 8%. Дело в том, что данных в тестовой выборке немного, около 50, а факторов много - 13 (или 14). Из-за этого R-sq adj существенно отличается от R-sq на тестовой выборке.

Общие выводы:

1. Если данных немного, то R-sq adj - более точная метрика качества прогнозирующей способности регрессии, так как она учтет количество факторов
2. Если данных много (существенно больше, чем факторов), нет большой разницы, что использовать - R-sq или R-sq adj, они будут почти одинаковыми
    

Важное замечание:

Все классные свойства коэффициентов детерминации работают только для моделей с intercept! В модели без intersept R-sq перестает показывать долю объясненной дисперсии.

Пример 2

Посмотрим, как визуально выглядят регрессии с большим и маленьким R-squared

In [0]:
print('Очень высокий коэффициент детерминации:')
# зададим случайным образом вектор признака а
a = np.random.randint(1,100, 50)

# зададим линейную зависимость между y и а
y=5+10*a+np.random.normal(0,7,len(a))

# преобразуем а для LinearRegression
adf=pd.DataFrame(a)

# прогоним регрессию
reg = LinearRegression(fit_intercept =True).fit(adf, y)

# найдем коэффициент детерминации
print('R-squared = ',reg.score(adf,y))

# нарисуем картинку: линия означает регрессионную прямую, точки - настоящие данные
y_pred=reg.predict(adf)
plt.scatter(a, y,  color='red')
plt.plot(a, y_pred, color='blue', linewidth=1)

In [0]:
print('Менее высокий коэффициент детерминации:')
#  добавим еще одну переменную
b = np.random.randint(-40,40, 50)

# усложним зависимость y от  а, b
y=5+10*a+5*b+np.random.normal(0,7,len(a))

# прогоним регрессию и выведем результаты
adf=pd.DataFrame(a)
reg = LinearRegression(fit_intercept =True).fit(adf, y)
print('R-squared = ',reg.score(adf,y))
y_pred=reg.predict(adf)
plt.scatter(a, y,  color='red')
plt.plot(a, y_pred, color='blue', linewidth=1)

Вывод:

1. Чем больше коэффициент детерминации, тем более кучно расположены точки вокруг регрессионной прямой, и, следовательно, тем выше качество предсказания

Пример 3

Интерполяция и экстраполяция  
Идея состоит в том, что в моделях простой линейной регрессии (не во временных рядах) мы можем хорошо прогнозировать ТОЛЬКО ВНУТРИ интервалов разброса признаков.

In [0]:
# создадим массив чисел от -0.4 до 2 с равным шагом 0.02
a = np.arange(-20,101)/50
# зададим теоретическую зависимость y(a) =  синус + случайная нормальная ошибка с нулевым средним и ст. отклонением 0.1
y = np.sin(a) + np.random.normal(0,0.1,len(a))

In [0]:
# разделим данные на две группы: группа inter - значения a от -0.2 до 0.8, группа extra - все остальные значения
a_inter = pd.DataFrame(a[0:60])
y_inter = y[0:60]
a_extra = pd.DataFrame(a[61:len(a)-1])
y_extra = y[61:len(a)-1]


In [0]:
# обучим регрессию на группе inter
reg = LinearRegression(fit_intercept =True).fit(a_inter, y_inter)
print('R-squared = ',reg.score(a_inter, y_inter))
y_pred=reg.predict(a_inter)
# нарисуем картинку
plt.scatter(a_inter, y_inter,  color='green')
plt.plot(a_inter, y_pred, color='black', linewidth=1)

In [0]:
# попробуем сделать предсказания на группе extra
y_pred_ex=reg.predict(a_extra)
plt.scatter(a_extra, y_extra,  color='green') # точки
plt.plot(a_extra, y_pred_ex, color='black', linewidth=1) # ожидание (предсказания модели)
plt.plot(a_extra, sin(a_extra), color='blue', linewidth=1) # реальность (настоящие данные)

На группе inter мы получили идеальную прямую с высоким R-sq. Как только попробовали сделать экстраполяцию - предсказания за пределами отрезка [-0.2, 0.8], модель резко перестала работать.

Почему так получилось?

Дело в том, что мы уверены в точности прогноза (благодяра высокому R-sq) ТОЛЬКО ВНУТРИ отрезка, на котором обучали регрессию - [-0.2,0.8] Что произойдет с теоретической зависимостью ЗА пределами этого отрезка, мы, находясь в нем, не знаем! В нашем случае на отрезке [-0.2, 0.8] синус ведет себя практически как линейная функция, что и дает нам красивую регрессию. Однако дальше он проявляет во всей красе свою нелинейность и модель больше не валидна.

Пример 4

Как можно понять, что линейная модель не подходит к данным?

1. Построить scatter plot (y,factor) - по нему можно предположить тренд (линейный, квадратичный, экспоненциальный итп)
2. Построить scatter plot остатков регрессионной модели - в нем не должно быть явных паттернов, тогда линейная модель годится

In [0]:
# создадим массив чисел от -0.2 до 2 с равным шагом 0.02
a = np.arange(-20,80)/50
# зададим теоретическую зависимость y(a) =  синус + случайная нормальная ошибка с нулевым средним и ст. отклонением 0.1
y = np.sin(a) + np.random.normal(0,0.1,len(a))

In [0]:
# способ 1
# по графику не очень ясно видно, есть нелинейность или нет 
plt.scatter(a, y,  color='black')

In [0]:
# Способ 2
# обучим все регрессию и посмотрим на картинку остатков
adf=pd.DataFrame(a)
reg = LinearRegression(fit_intercept =True).fit(adf, y)
print('R-squared = ',reg.score(adf, y))
y_pred = reg.predict(adf)
residuals = y - y_pred
# нарисуем картинку
zero = np.zeros(len(a))
plt.scatter(adf, residuals,  color='red')
plt.plot(adf,zero, color='green', linewidth=1)

Мы видим, что в левой и правой части графика остатков почти все точки расположены ниже нулевой отметки, а в середине - выше.

Это и есть закономерность - нелинейный паттерн. Линейная модель неприменима